In [290]:
from sklearn.datasets import fetch_lfw_people
getimgtr = fetch_lfw_people(min_faces_per_person=15, color=True, resize=1)

/home/cc/.local/lib/python3.5/site-packages/sklearn/datasets/lfw.py:195: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  face = imresize(face, resize)
/home/cc/.local/lib/python3.5/site-packages/sklearn/datasets/lfw.py:195: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  face = imresize(face, resize)
/home/cc/.local/lib/python3.5/site-packages/sklearn/datasets/lfw.py:195: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  face = imresize(face, resize)
/home/cc/.local/lib/python3.5/site-packages/sklearn/datasets/lfw.py:195: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.tran

In [291]:
n_samples, h1,w1,ch=getimgtr.images.shape


#defining varibles
X1=getimgtr.data
n_features1=X1.shape[1]
y1=getimgtr.target
target_names1=getimgtr.target_names
n_classes1=target_names1.shape[0]

In [292]:
from sklearn.model_selection import train_test_split

X_training, X_testing, y_training, y_testing =train_test_split(X1,y1, test_size=.20, random_state=42)

In [293]:
from keras.utils import to_categorical
train_Y_one_hot1 = to_categorical(y_training)
test_Y_one_hot1 = to_categorical(y_testing)

In [294]:
X_train1=X_training.reshape(-1, h1,w1, ch)
X_test1=X_testing.reshape(-1,h1,w1,ch)

X_train1=X_train1.astype('float32')
X_test1=X_test1.astype('float32')

X_train1=X_train1/255
X_test1=X_test1/255

X_train1.shape, X_test1.shape, train_Y_one_hot1.shape, test_Y_one_hot1.shape

((2876, 125, 94, 3), (719, 125, 94, 3), (2876, 96), (719, 96))

In [283]:
hidden_dim=512

In [295]:
import keras
from keras.models import Sequential,Input,Model
from keras import layers
from keras.layers import Dense, Flatten,Input,Dropout,Activation,Conv2D

In [285]:
from keras_vggface.vggface import VGGFace

In [286]:
base_model = VGGFace(include_top=False, input_shape=(125, 94, 3), pooling='max')


In [287]:
for layer in base_model.layers:
    layer.trainable = False

In [296]:

last_layer = base_model.get_layer('pool5').output
x = Flatten(name='flatten')(last_layer)
#x = Dense(hidden_dim, activation='relu')(x)
x=Dropout(0.5)(x)
x = Dense(hidden_dim, activation='relu')(x)
out = Dense(n_classes1, activation='softmax')(x)
custom_vgg_model = Model(base_model.input, out)

In [297]:
batch_size = 64
epochs = 20

custom_vgg_model.compile(loss=keras.losses.categorical_crossentropy,
         optimizer=keras.optimizers.Adam(),metrics=['accuracy'])


custom_vgg_model.fit(X_train1, train_Y_one_hot1, 
     batch_size=batch_size,
     epochs=epochs,verbose=1,
     
     validation_data=(X_test1, test_Y_one_hot1))

Train on 2876 samples, validate on 719 samples
Epoch 1/20
2876/2876 [==============================] - 40s 14ms/step - loss: 2.8890 - acc: 0.4172 - val_loss: 1.6732 - val_acc: 0.6857
Epoch 2/20
2876/2876 [==============================] - 40s 14ms/step - loss: 1.0073 - acc: 0.8390 - val_loss: 0.6567 - val_acc: 0.8999
Epoch 3/20
2876/2876 [==============================] - 39s 14ms/step - loss: 0.4203 - acc: 0.9506 - val_loss: 0.3826 - val_acc: 0.9388
Epoch 4/20
2876/2876 [==============================] - 39s 14ms/step - loss: 0.2309 - acc: 0.9788 - val_loss: 0.2737 - val_acc: 0.9583
Epoch 5/20
2876/2876 [==============================] - 39s 14ms/step - loss: 0.1480 - acc: 0.9882 - val_loss: 0.2330 - val_acc: 0.9541
Epoch 6/20
2876/2876 [==============================] - 39s 14ms/step - loss: 0.1053 - acc: 0.9934 - val_loss: 0.2050 - val_acc: 0.9624
Epoch 7/20
2876/2876 [==============================] - 39s 14ms/step - loss: 0.0774 - acc: 0.9962 - val_loss: 0.1882 - val_acc: 0.9583
E

In [276]:
custom_vgg_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [253]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint


In [299]:
train_datagen = ImageDataGenerator(
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator()
 
train_batchsize = 100
val_batchsize = 10
 
train_generator = train_datagen.flow(
        X_train1, train_Y_one_hot1, batch_size=train_batchsize)
 
validation_generator = validation_datagen.flow(
        X_test1,test_Y_one_hot1,
        batch_size=val_batchsize,
        shuffle=False)

train_datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [255]:
from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(monitor='val_acc', patience=2)

In [300]:
history = custom_vgg_model.fit_generator(
      train_generator,
      steps_per_epoch=X_train1.shape[0] // batch_size,
      epochs=50,
      validation_data=validation_generator,
      
    callbacks=[ModelCheckpoint('VGG16-transferlearning.model', monitor='val_acc', save_best_only=True),
              early_stopping_monitor])

Epoch 1/50
44/44 [==============================] - 57s 1s/step - loss: 0.5092 - acc: 0.8723 - val_loss: 0.1376 - val_acc: 0.9597
Epoch 2/50
44/44 [==============================] - 56s 1s/step - loss: 0.3792 - acc: 0.9022 - val_loss: 0.1272 - val_acc: 0.9555
Epoch 3/50
44/44 [==============================] - 57s 1s/step - loss: 0.2973 - acc: 0.9265 - val_loss: 0.1146 - val_acc: 0.9694
Epoch 4/50
44/44 [==============================] - 56s 1s/step - loss: 0.2843 - acc: 0.9276 - val_loss: 0.1042 - val_acc: 0.9680
Epoch 5/50
44/44 [==============================] - 56s 1s/step - loss: 0.2490 - acc: 0.9328 - val_loss: 0.0984 - val_acc: 0.9694
